In [11]:
import mindspore.nn as nn
import mindspore as ms
from mindspore import dtype as mstype
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as C
import mindspore.dataset.transforms.c_transforms as C2
from mindspore import context
import numpy as np
import matplotlib.pyplot as plt
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
from mindspore import load_checkpoint, load_param_into_net
import os
from mindspore import Model
from mindspore import Tensor  # mindspore的tensor

import moxing as mox
mox.file.copy_parallel(src_url="s3://nlp-zju-cxy/nlp/vqa/data/images/train", dst_url='./train')
mox.file.copy_parallel(src_url="s3://nlp-zju-cxy/nlp/vqa/CNN/resnet.py", dst_url='./resnet.py')
mox.file.copy_parallel(src_url="s3://nlp-zju-cxy/nlp/resnet.ckpt", dst_url='./resnet.ckpt')

context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")

INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing OBS: 32000
INFO:root:pid: None.	1000/32013
INFO:root:pid: None.	2000/32013
INFO:root:pid: No

In [12]:
from resnet import ImtNet, ResidualBlock
net = ImtNet(ResidualBlock, batch_size=32, num_classes=10)
paramDict = load_checkpoint('./resnet.ckpt', net)

In [13]:
import cv2 as cv
import os
import copy
import mindspore.ops as op
import pickle as pkl

i = 0
stack = op.Stack()
array = []
outlist = []
count = 0
perm = (2, 0, 1)
trans = op.Transpose()
pics = {}
filenames = ""
for filename in os.listdir('./train'):
    count = count + 1
    if count % 1000 == 0:
        print('Processed: ', count)
    img = cv.imread('./train'+'/'+filename)
    img = cv.resize(img,(224,224))
    input = Tensor(img,ms.float32)
    input = trans(input, perm)
    filenames = filenames + filename
    if i < 32:
        array.append(input)
    else:
        out = stack(array)
#         print(123)
        out = net(out)
#         print(123)
#         out = np.array(out)
#         print(123)
        pics[filenames] = out
        filenames = ""
        array = []
        array.append(input)
        i = 0
    i = i + 1
if i > 0:
    while i < 32:
        i = i + 1
        array.append(Tensor(np.zeros([3,224,224]),ms.float32))
    out = stack(array)
    out = net(out)
#     out = np.array(out)
    pics[filenames] = out

Processed:  1000
Processed:  2000
Processed:  3000
Processed:  4000
Processed:  5000
Processed:  6000
Processed:  7000
Processed:  8000
Processed:  9000
Processed:  10000
Processed:  11000
Processed:  12000
Processed:  13000
Processed:  14000
Processed:  15000
Processed:  16000
Processed:  17000
Processed:  18000
Processed:  19000
Processed:  20000
Processed:  21000
Processed:  22000
Processed:  23000
Processed:  24000
Processed:  25000
Processed:  26000
Processed:  27000
Processed:  28000
Processed:  29000
Processed:  30000
Processed:  31000
Processed:  32000


In [14]:
# print(pics)
pkl.dump(pics,open("train_feat.pkl","wb"))

In [15]:
mox.file.copy_parallel(src_url="./train_feat.pkl", dst_url='s3://nlp-zju-cxy/nlp/vqa/train_feat.pkl')

In [16]:
d = pkl.load(open("val_feat.pkl", "rb"))
print(len(d))

755
